In [40]:
%%bash
pip3 install psycopg2-binary
pip3 install networkx
pip3 install colorama
pip3 install termcolor

In [67]:
import subprocess
import psycopg2
from pathlib import Path
import json
import ctdPython
from functools import partial
from ctdPython.hypergraph import HyperGraph
from ctdPython.ctdcheck import CTDCheck
from ctdPython.ctdcheck import Block
from ctdPython.ctdcheck import VertSet

In [68]:
def all_choose_k(S, k):
    from itertools import chain, combinations
    return chain(*(combinations(S, kp) for kp in range(1,k+1)))
#
def all_lambdas(E, k):
    for es in all_choose_k(E, k):
        yield set.union(*es)

# (over approximates) the bags produced by the LogK algorithm
def computesoftk(h, k):
    softk = list()
    for P in all_lambdas(h.E,k):
        for C in h.separate(P, only_vertices=True):
            for L in all_lambdas(h.E, k):
                B = set.intersection(C, L)
                if len(B) > 1 and B not in softk:
                    softk.append(B)
    return softk
    
# computes the blocks of a bag by computing its components w.r.t. h
def bag_to_blocks(h,B):  
    blocks = list()
    for C in h.separate(B, only_vertices=True):
        blocks.append(Block(VertSet(B),VertSet(C)))
    blocks.append(Block(VertSet(B),VertSet(set())))  # adding trivial block too
    return blocks


# computes the blocks of a bag by computing its components w.r.t. h
def bag_to_blocksConnected(h,B):  
    blocks = list()
    for C in h.separate(B, only_vertices=True):
        tempBlock = Block(VertSet(B),VertSet(C))
        if tempBlock.connected(h):
            blocks.append(tempBlock)
    blocks.append(Block(VertSet(B),VertSet(set())))  # adding trivial block too
    return blocks


# Same as  computeosftK, but returns directly the blocks
def computesoftkBlocks(h, k):
    out = list()
    listOfLists = map(partial(bag_to_blocks,h),computesoftk(h,k))
    for ll in listOfLists:
        for l in ll:
            out.append(l)
    return out


# Same as  computeosftK, but returns directly the blocks
def computesoftkBlocksConnected(h, k):
    out = list()
    listOfLists = map(partial(bag_to_blocksConnected,h),computesoftk(h,k))
    for ll in listOfLists:
        for l in ll:
            out.append(l)
    return out

In [69]:
REWRITE_JAR = 'rewrite-assembly-0.1.0-SNAPSHOT.jar'

class Rewriting:
    def __init__(self, original, rewritten, features, time, drop_statements):
        self.original = original
        self.rewritten = rewritten
        self.features = features
        self.time = time
        self.drop_statements = drop_statements

class QueryRewriter:
    def __init__(self, host, database, user, password, port=5432):
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.port = port
        self.jdbcString = f'jdbc:postgresql://{self.host}:{self.port}/{self.database}'

    def rewrite(self, query):
        rewriteProcess = subprocess.run(['java', '-jar', REWRITE_JAR, query,
                                         self.jdbcString, self.database, self.user, self.password])
        print(rewriteProcess.returncode)
        if (rewriteProcess.returncode != 0):
            print(f'Error calling {rewriteProcess}')
            return None

        output = json.loads(Path('output/output.json').read_text())
        drop_output = json.loads(Path('output/drop.json').read_text())

        result = Rewriting(query, output['rewritten_query'], output['features'], output['time'],
                           drop_output['rewritten_query'])

        hg = HyperGraph.fromHyperbench('output/hypergraph.txt')

        acyclic = output['acyclic']
        print('acyclic: ' + str(acyclic))
        print('hg: ' + str(hg))
        ctd = CTDCheck(hg)
        print('ctd: ' + str(ctd))
        blocks = computesoftkBlocksConnected(hg,2)

        for b in blocks:
            print("Adding blocks ", b)
            ctd.addBlock(b)
            # headedBy = ctd.head_to_blocks[repr(b.head)]
            # print("All blocks headed by ",b.head )
            # for h in headedBy:
                # print(h)

        res = ctd.hasDecomp()
        print("hasDecomp done. Result: ",res)
        
        return result

query = """
SELECT MIN(p.Id) FROM posts as p, postLinks as pl, postHistory as ph, votes as v, badges as b, users as u WHERE p.Id = pl.RelatedPostId AND u.Id = p.OwnerUserId AND u.Id = b.UserId AND u.Id = ph.UserId AND u.Id = v.UserId AND p.AnswerCount>=0 AND p.FavoriteCount>=0 AND pl.LinkTypeId=1 AND ph.PostHistoryTypeId=2 AND v.CreationDate>=CAST('2010-07-20 00:00:00' AS TIMESTAMP) AND u.Reputation>=1 AND u.DownVotes>=0 AND u.DownVotes<=0 AND u.UpVotes<=439 AND u.CreationDate<=CAST('2014-08-07 11:18:45' AS TIMESTAMP)
"""

rewriter = QueryRewriter('postgres', 'stats', 'stats', 'stats')

result = rewriter.rewrite(query)
print(result)

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


org.apache.calcite.jdbc.CalciteSchema$SchemaPlusImpl@771d1ffbLogicalAggregate(group=[{}], EXPR$0=[MIN($0)])
  LogicalProject(id=[$0])
    LogicalJoin(condition=[=($5, $29)], joinType=[inner])
      LogicalJoin(condition=[=($5, $27)], joinType=[inner])
        LogicalJoin(condition=[=($5, $24)], joinType=[inner])
          LogicalJoin(condition=[=($5, $19)], joinType=[inner])
            LogicalJoin(condition=[=($0, $13)], joinType=[inner])
              LogicalFilter(condition=[AND(>=($6, 0), >=($8, 0))])
                JdbcTableScan(table=[[stats, posts]])
              LogicalFilter(condition=[=(CAST($4):INTEGER, 1)])
                JdbcTableScan(table=[[stats, postlinks]])
            LogicalFilter(condition=[=(CAST($1):INTEGER, 2)])
              JdbcTableScan(table=[[stats, posthistory]])
          LogicalFilter(condition=[>=($3, 2010-07-20 00:00:00)])
            JdbcTableScan(table=[[stats, votes]])
        JdbcTableScan(table=[[stats, badges]])
      LogicalFilter(condition=[

In [71]:
query = """
SELECT MIN(pkp1.Person1Id)
FROM City AS CityA
JOIN City AS CityB
  ON CityB.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN City AS CityC
  ON CityC.isPartOf_CountryId = CityA.isPartOf_CountryId
JOIN Person AS PersonA
  ON PersonA.isLocatedIn_CityId = CityA.CityId
JOIN Person AS PersonB
  ON PersonB.isLocatedIn_CityId = CityB.CityId
JOIN Person AS PersonC
  ON PersonC.isLocatedIn_CityId = CityC.CityId
JOIN Person_knows_Person AS pkp1
  ON pkp1.Person1Id = PersonA.PersonId
 AND pkp1.Person2Id = PersonB.PersonId
JOIN Person_knows_Person AS pkp2
  ON pkp2.Person1Id = PersonB.PersonId
 AND pkp2.Person2Id = PersonC.PersonId
JOIN Person_knows_Person AS pkp3
  ON pkp3.Person1Id = PersonC.PersonId
 AND pkp3.Person2Id = PersonA.PersonId

"""

rewriter = QueryRewriter('postgres', 'lsqb', 'lsqb', 'lsqb')

result = rewriter.rewrite(query)
print(result)

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


ERROR StatusLogger Log4j2 could not find a logging implementation. Please add log4j-core to the classpath. Using SimpleLogger to log to the console...


org.apache.calcite.jdbc.CalciteSchema$SchemaPlusImpl@771d1ffbLogicalAggregate(group=[{}], EXPR$0=[MIN($0)])
  LogicalProject(person1id=[$12])
    LogicalJoin(condition=[AND(=($16, $10), =($17, $6))], joinType=[inner])
      LogicalJoin(condition=[AND(=($14, $8), =($15, $10))], joinType=[inner])
        LogicalJoin(condition=[AND(=($12, $6), =($13, $8))], joinType=[inner])
          LogicalJoin(condition=[=($11, $4)], joinType=[inner])
            LogicalJoin(condition=[=($9, $2)], joinType=[inner])
              LogicalJoin(condition=[=($7, $0)], joinType=[inner])
                LogicalJoin(condition=[=($5, $1)], joinType=[inner])
                  LogicalJoin(condition=[=($3, $1)], joinType=[inner])
                    JdbcTableScan(table=[[lsqb, city]])
                    JdbcTableScan(table=[[lsqb, city]])
                  JdbcTableScan(table=[[lsqb, city]])
                JdbcTableScan(table=[[lsqb, person]])
              JdbcTableScan(table=[[lsqb, person]])
            JdbcT

In [ ]:
def run_rewritten(query):
    conn = psycopg2.connect(
        host="postgres",
        database=database,
        user=database,
        password=database
    )